In [10]:
#pip install datasets

import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset, DatasetDict
from transformers.keras_callbacks import KerasMetricCallback

def batch_predict_sentiment(texts, tokenizer, model, batch_size=10):
    total_texts = len(texts)
    predicted_sentiments = []

    for i in range(0, total_texts, batch_size):
        texts_batch = texts[i:i + batch_size]
        inputs = tokenizer(texts_batch, return_tensors="pt", padding=True, truncation=True)

        print(f"Predicting batch {i // batch_size + 1}/{(total_texts + batch_size - 1) // batch_size}")

        with torch.no_grad():
            outputs = model(**inputs)

        predicted_classes = torch.argmax(outputs.logits, dim=1).tolist()
        predicted_sentiments.extend(predicted_classes)

    return predicted_sentiments

def calculate_accuracy(predicted_sentiments, actual_labels):
    correct_predictions = sum(1 for pred, label in zip(predicted_sentiments, actual_labels) if pred == label)
    total_predictions = len(actual_labels)
    accuracy = correct_predictions / total_predictions * 100
    return accuracy

def predict_model(test_data, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    batch_predicted_sentiments = batch_predict_sentiment(test_data['text'], tokenizer, model)
    actual_labels = test_data['label']

    accuracy = calculate_accuracy(batch_predicted_sentiments, actual_labels)
    print(f"The accuracy achieved for predictions using {model_name} is {accuracy:.2f}%")
    return accuracy

def main():
    imdb = load_dataset("imdb")
    small_test_dataset = imdb["test"].shuffle(seed=42).select(range(200))

    # Predict using GPT2
    gpt2_accuracy = predict_model(small_test_dataset, "mnoukhov/gpt2-imdb-sentiment-classifier")

    # Predict using RoBERTa
    roberta_accuracy = predict_model(small_test_dataset, "abhishek/autonlp-imdb-roberta-base-3662644")

    # Predict using BERT
    bert_accuracy = predict_model(small_test_dataset, "bert-base-uncased")

    # Comparing accuracies
    print("\nComparing accuracies:")
    print(f"GPT2 Accuracy: {gpt2_accuracy:.2f}%")
    print(f"RoBERTa Accuracy: {roberta_accuracy:.2f}%")
    print(f"BERT Accuracy: {bert_accuracy:.2f}%")

if __name__ == '__main__':
    main()


Predicting batch 1/20
Predicting batch 2/20
Predicting batch 3/20
Predicting batch 4/20
Predicting batch 5/20
Predicting batch 6/20
Predicting batch 7/20
Predicting batch 8/20
Predicting batch 9/20
Predicting batch 10/20
Predicting batch 11/20
Predicting batch 12/20
Predicting batch 13/20
Predicting batch 14/20
Predicting batch 15/20
Predicting batch 16/20
Predicting batch 17/20
Predicting batch 18/20
Predicting batch 19/20
Predicting batch 20/20
The accuracy achieved for predictions using mnoukhov/gpt2-imdb-sentiment-classifier is 91.00%
Predicting batch 1/20
Predicting batch 2/20
Predicting batch 3/20
Predicting batch 4/20
Predicting batch 5/20
Predicting batch 6/20
Predicting batch 7/20
Predicting batch 8/20
Predicting batch 9/20
Predicting batch 10/20
Predicting batch 11/20
Predicting batch 12/20
Predicting batch 13/20
Predicting batch 14/20
Predicting batch 15/20
Predicting batch 16/20
Predicting batch 17/20
Predicting batch 18/20
Predicting batch 19/20
Predicting batch 20/20
The 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicting batch 1/20
Predicting batch 2/20
Predicting batch 3/20
Predicting batch 4/20
Predicting batch 5/20
Predicting batch 6/20
Predicting batch 7/20
Predicting batch 8/20
Predicting batch 9/20
Predicting batch 10/20
Predicting batch 11/20
Predicting batch 12/20
Predicting batch 13/20
Predicting batch 14/20
Predicting batch 15/20
Predicting batch 16/20
Predicting batch 17/20
Predicting batch 18/20
Predicting batch 19/20
Predicting batch 20/20
The accuracy achieved for predictions using bert-base-uncased is 50.00%

Comparing accuracies:
GPT2 Accuracy: 91.00%
RoBERTa Accuracy: 93.00%
BERT Accuracy: 50.00%
